<a href="https://colab.research.google.com/github/Ahmad162539/Smartbuilding/blob/main/Primary_Classroom_Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load data

In [1]:
import pandas as pd

url_primclass= 'https://raw.githubusercontent.com/Ahmad162539/Smartbuilding/main/data%20of%20PrimClass_Jaden.csv'
primclass= pd.read_csv(url_primclass)
data_primclass= primclass[(primclass['timestamp'] > '2015-03-01') & (primclass['timestamp'] < '2015-06-01')]
dfs_primclass=data_primclass['energy']
datas_primclass=pd.DataFrame(dfs_primclass)


## import libraries

In [2]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter('ignore')

import numpy
import math
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn import metrics

import time

In [3]:
pip install EMD-signal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0


In [4]:
from PyEMD import CEEMDAN

In [5]:
pip install dataframe_image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 kB 26.7 MB/s eta 0:00:00


In [6]:
import dataframe_image as dfi

In [7]:
#Parameters Settings

max_features=8 # the feature number of each node is set to 8, as it is suggested to be one-third of the feature's number (Lahouar and Slama,2017)
lr=0.001 #learning rate LSTM as recommended in (Kingma and Ba, 2014)
optimizer='Adam' #Adam is used as the optimizer for LSTM as it is computationally efficient (Dubey, Ashutosh Kumar, et al,2021)
neuron=64
epoch=100 #(Jorges et al, 2021)
batch_size=64 #(Kandel et al,2020)
hours=24 #in this study, to predict one hour ahead predicton, we use input from the previous 24 hour energy consumption
data_partition=0.8 #in this study, we divided the data into 80% of data as training data and 20% of the data as testing data

## Import all functions from another notebook for building prediction method

In [8]:
import numpy
import pandas as pd
import tensorflow as tf
from PyEMD import CEEMDAN
import warnings
warnings.filterwarnings("ignore")


### import the libraries
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from math import sqrt

# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

def percentage_error(actual, predicted):
    res = numpy.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred):
    return numpy.mean(numpy.abs(percentage_error(numpy.asarray(y_true), numpy.asarray(y_pred)))) * 100



# In[25]:


def lr_model(datass,look_back,data_partition):

    datasets=datass.values
    train_size = int(len(datasets) * data_partition)
    test_size = len(datasets) - train_size
    train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)
    sc_X = StandardScaler()
    sc_y = StandardScaler()

    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()
    import tensorflow as tf

    numpy.random.seed(1234)
    tf.random.set_seed(1234)

    from sklearn.linear_model import LinearRegression

    grid = LinearRegression()

    grid.fit(X,y)
    y_pred_train_lr= grid.predict(X)
    y_pred_test_lr= grid.predict(X1)

    y_pred_train_lr=pd.DataFrame(y_pred_train_lr)
    y_pred_test_lr=pd.DataFrame(y_pred_test_lr)

    y1=pd.DataFrame(y1)
    y=pd.DataFrame(y)


    y_pred_test1_lr= sc_y.inverse_transform (y_pred_test_lr)
    y_pred_train1_lr=sc_y.inverse_transform (y_pred_train_lr)

    y_test= sc_y.inverse_transform (y1)
    y_train= sc_y.inverse_transform (y)

    y_pred_test1_rf=pd.DataFrame(y_pred_test1_lr)
    y_pred_train1_rf=pd.DataFrame(y_pred_train1_lr)

    y_test= pd.DataFrame(y_test)


    #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,y_pred_test1_lr)
    rmse= sqrt(mean_squared_error(y_test,y_pred_test1_lr))
    mae=metrics.mean_absolute_error(y_test,y_pred_test1_lr)


    return mape,rmse,mae


# In[26]:


def svr_model(datass,look_back,data_partition):

    datasets=datass.values
    train_size = int(len(datasets) * data_partition)
    test_size = len(datasets) - train_size
    train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)
    sc_X = StandardScaler()
    sc_y = StandardScaler()

    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()

    numpy.random.seed(1234)

    import tensorflow as tf
    tf.random.set_seed(1234)


    from sklearn.svm import SVR

    grid = SVR()
    grid.fit(X,y)
    y_pred_train_svr= grid.predict(X)
    y_pred_test_svr= grid.predict(X1)

    y_pred_train_svr=pd.DataFrame(y_pred_train_svr)
    y_pred_test_svr=pd.DataFrame(y_pred_test_svr)

    y1=pd.DataFrame(y1)
    y=pd.DataFrame(y)


    y_pred_test1_svr= sc_y.inverse_transform (y_pred_test_svr)
    y_pred_train1_svr=sc_y.inverse_transform (y_pred_train_svr)

    y_test= sc_y.inverse_transform (y1)
    y_train= sc_y.inverse_transform (y)

    y_pred_test1_svr=pd.DataFrame(y_pred_test1_svr)
    y_pred_train1_svr=pd.DataFrame(y_pred_train1_svr)

    y_test= pd.DataFrame(y_test)


    #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,y_pred_test1_svr)
    rmse= sqrt(mean_squared_error(y_test,y_pred_test1_svr))
    mae=metrics.mean_absolute_error(y_test,y_pred_test1_svr)


    return mape,rmse,mae


# In[27]:



def ann_model(datass,look_back,data_partition):


    datasets=datass.values
    train_size = int(len(datasets) * data_partition)
    test_size = len(datasets) - train_size
    train, test = datasets[0:train_size], datasets[train_size:len(datasets)]
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)
    sc_X = StandardScaler()
    sc_y = StandardScaler()

    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()

    import numpy

    trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
    testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))


    numpy.random.seed(1234)
    import tensorflow as tf
    tf.random.set_seed(1234)


    from sklearn.neural_network import MLPRegressor

    model= MLPRegressor(random_state=1,activation='tanh').fit(X,y)

    numpy.random.seed(1234)


    # make predictions
    y_pred_train = model.predict(X)
    y_pred_test = model.predict(X1)
    y_pred_test= numpy.array(y_pred_test).ravel()

    y_pred_test=pd.DataFrame(y_pred_test)
    y1=pd.DataFrame(y1)

    y_pred_test1= sc_y.inverse_transform (y_pred_test)
    y_test= sc_y.inverse_transform (y1)







    #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,y_pred_test1)
    rmse= sqrt(mean_squared_error(y_test,y_pred_test1))
    mae=metrics.mean_absolute_error(y_test,y_pred_test1)


    return mape,rmse,mae


# In[28]:


def rf_model(datass,look_back,data_partition,max_features):

    datasets=datass.values
    train_size = int(len(datasets) * data_partition)
    test_size = len(datasets) - train_size
    train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)
    sc_X = StandardScaler()
    sc_y = StandardScaler()

    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()

    numpy.random.seed(1234)
    import tensorflow as tf
    tf.random.set_seed(1234)

    from sklearn.ensemble import RandomForestRegressor

    grid = RandomForestRegressor(max_features=max_features)
    grid.fit(X,y)
    y_pred_train_rf= grid.predict(X)
    y_pred_test_rf= grid.predict(X1)

    y_pred_train_rf=pd.DataFrame(y_pred_train_rf)
    y_pred_test_rf=pd.DataFrame(y_pred_test_rf)

    y1=pd.DataFrame(y1)
    y=pd.DataFrame(y)


    y_pred_test1_rf= sc_y.inverse_transform (y_pred_test_rf)
    y_pred_train1_rf=sc_y.inverse_transform (y_pred_train_rf)

    y_test= sc_y.inverse_transform (y1)
    y_train= sc_y.inverse_transform (y)

    y_pred_test1_rf=pd.DataFrame(y_pred_test1_rf)
    y_pred_train1_rf=pd.DataFrame(y_pred_train1_rf)

    y_test= pd.DataFrame(y_test)


    #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,y_pred_test1_rf)
    rmse= sqrt(mean_squared_error(y_test,y_pred_test1_rf))
    mae=metrics.mean_absolute_error(y_test,y_pred_test1_rf)


    return mape,rmse,mae


# In[29]:


def lstm_model(datass,look_back,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer):
    datasets=datass.values
    train_size = int(len(datasets) * data_partition)
    test_size = len(datasets) - train_size
    train, test = datasets[0:train_size], datasets[train_size:len(datasets)]
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)
    sc_X = StandardScaler()
    sc_y = StandardScaler()

    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()
    trainX1 = numpy.reshape(X, (X.shape[0],1,X.shape[1]))
    testX1 = numpy.reshape(X1, (X1.shape[0],1,X1.shape[1]))

    numpy.random.seed(1234)
    import tensorflow as tf
    tf.random.set_seed(1234)

    import os
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    from keras.models import Sequential
    from keras.layers.core import Dense, Dropout, Activation
    from keras.layers import LSTM


    neuron=neuron
    model = Sequential()
    model.add(LSTM(units = neuron,input_shape=(trainX1.shape[1], trainX1.shape[2])))
    model.add(Dense(1))
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss='mse',optimizer=optimizer)
#    model.summary()


  # Fitting the RNN to the Training s
    model.fit(trainX1, y, epochs = epoch, batch_size = batch_size,verbose=0)
  # make predictions
    y_pred_train = model.predict(trainX1)
    y_pred_test = model.predict(testX1)
    y_pred_test= numpy.array(y_pred_test).ravel()

    y_pred_test=pd.DataFrame(y_pred_test)
    y_pred_test1= sc_y.inverse_transform (y_pred_test)
    y1=pd.DataFrame(y1)

    y_test= sc_y.inverse_transform (y1)

    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import mean_absolute_error

    from sklearn import metrics

    mape=mean_absolute_percentage_error(y_test,y_pred_test1)
    rmse= sqrt(mean_squared_error(y_test,y_pred_test1))
    mae=metrics.mean_absolute_error(y_test,y_pred_test1)
    return mape,rmse,mae


# In[30]:


###################################################hybrid based ceemdan####################################################
def hybrid_ceemdan_rf(datass,look_back,data_partition,max_features):


    import numpy as np
    import pandas as pd

    dfs=datass
    s = dfs.values

    emd = CEEMDAN(epsilon=0.05)
    emd.noise_seed(12345)

    IMFs = emd(s)


    full_imf=pd.DataFrame(IMFs)
    data_imf=full_imf.T

    import pandas as pd

    pred_test=[]
    test_ori=[]
    pred_train=[]
    train_ori=[]


    for col in data_imf:

        datasetss2=pd.DataFrame(data_imf[col])
        datasets=datasetss2.values
        train_size = int(len(datasets) * data_partition)
        test_size = len(datasets) - train_size
        train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

        trainX, trainY = create_dataset(train, look_back)
        testX, testY = create_dataset(test, look_back)
        X_train=pd.DataFrame(trainX)
        Y_train=pd.DataFrame(trainY)
        X_test=pd.DataFrame(testX)
        Y_test=pd.DataFrame(testY)
        sc_X = StandardScaler()
        sc_y = StandardScaler()
        X= sc_X.fit_transform(X_train)
        y= sc_y.fit_transform(Y_train)
        X1= sc_X.fit_transform(X_test)
        y1= sc_y.fit_transform(Y_test)
        y=y.ravel()
        y1=y1.ravel()

        import numpy

        numpy.random.seed(1234)
        import tensorflow as tf
        tf.random.set_seed(1234)


        from sklearn.ensemble import RandomForestRegressor


        grid = RandomForestRegressor(max_features=max_features)
        grid.fit(X,y)
        y_pred_train= grid.predict(X)
        y_pred_test= grid.predict(X1)

        y_pred_test=pd.DataFrame(y_pred_test)
        y_pred_train=pd.DataFrame(y_pred_train)

        y1=pd.DataFrame(y1)
        y=pd.DataFrame(y)

        y_test= sc_y.inverse_transform (y1)
        y_train= sc_y.inverse_transform (y)

        y_pred_test1= sc_y.inverse_transform (y_pred_test)
        y_pred_train1= sc_y.inverse_transform (y_pred_train)


        pred_test.append(y_pred_test1)
        test_ori.append(y_test)
        pred_train.append(y_pred_train1)
        train_ori.append(y_train)


    result_pred_test= pd.DataFrame.from_records(pred_test)
    result_pred_train= pd.DataFrame.from_records(pred_train)


    a=result_pred_test.sum(axis = 0, skipna = True)
    b=result_pred_train.sum(axis = 0, skipna = True)


    dataframe=pd.DataFrame(dfs)
    dataset=dataframe.values

    train_size = int(len(dataset) * data_partition)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size], dataset[train_size:len(dataset)]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)

    sc_X = StandardScaler()
    sc_y = StandardScaler()
    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()


    trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
    testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))

    numpy.random.seed(1234)
    import tensorflow as tf
    tf.random.set_seed(1234)

    y1=pd.DataFrame(y1)
    y=pd.DataFrame(y)
    y_test= sc_y.inverse_transform (y1)
    y_train= sc_y.inverse_transform (y)




    a= pd.DataFrame(a)
    y_test= pd.DataFrame(y_test)




   #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,a)
    rmse= sqrt(mean_squared_error(y_test,a))
    mae=metrics.mean_absolute_error(y_test,a)



    return mape,rmse,mae


# In[31]:


def hybrid_ceemdan_lstm(datass,look_back,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer):


    from PyEMD import CEEMDAN

    dfs=datass
    s = dfs.values

    emd = CEEMDAN(epsilon=0.05)
    emd.noise_seed(12345)

    IMFs = emd(s)


    full_imf=pd.DataFrame(IMFs)
    data_imf=full_imf.T

    pred_test=[]
    test_ori=[]
    pred_train=[]
    train_ori=[]


    for col in data_imf:

        datasetss2=pd.DataFrame(data_imf[col])
        datasets=datasetss2.values
        train_size = int(len(datasets) * data_partition)
        test_size = len(datasets) - train_size
        train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

        trainX, trainY = create_dataset(train, look_back)
        testX, testY = create_dataset(test, look_back)
        X_train=pd.DataFrame(trainX)
        Y_train=pd.DataFrame(trainY)
        X_test=pd.DataFrame(testX)
        Y_test=pd.DataFrame(testY)
        sc_X = StandardScaler()
        sc_y = StandardScaler()
        X= sc_X.fit_transform(X_train)
        y= sc_y.fit_transform(Y_train)
        X1= sc_X.fit_transform(X_test)
        y1= sc_y.fit_transform(Y_test)
        y=y.ravel()
        y1=y1.ravel()

        import numpy

        trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
        testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))

        numpy.random.seed(1234)
        import tensorflow as tf
        tf.random.set_seed(1234)


        from keras.models import Sequential
        from keras.layers.core import Dense, Dropout, Activation
        from keras.layers import LSTM


        neuron=neuron

        model = Sequential()
        model.add(LSTM(units = neuron,input_shape=(trainX.shape[1], trainX.shape[2])))
        model.add(Dense(1))
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(loss='mse',optimizer=optimizer)

        numpy.random.seed(1234)


        # Fitting the RNN to the Training set
        model.fit(trainX, y, epochs = epoch, batch_size = batch_size,verbose=0)

        # make predictions
        y_pred_train = model.predict(trainX)
        y_pred_test = model.predict(testX)

        # make predictions

        y_pred_test= numpy.array(y_pred_test).ravel()
        y_pred_test=pd.DataFrame(y_pred_test)
        y1=pd.DataFrame(y1)
        y=pd.DataFrame(y)
        y_pred_train= numpy.array(y_pred_train).ravel()
        y_pred_train=pd.DataFrame(y_pred_train)


        y_test= sc_y.inverse_transform (y1)
        y_train= sc_y.inverse_transform (y)

        y_pred_test1= sc_y.inverse_transform (y_pred_test)
        y_pred_train1= sc_y.inverse_transform (y_pred_train)


        pred_test.append(y_pred_test1)
        test_ori.append(y_test)
        pred_train.append(y_pred_train1)
        train_ori.append(y_train)


    result_pred_test= pd.DataFrame.from_records(pred_test)
    result_pred_train= pd.DataFrame.from_records(pred_train)


    a=result_pred_test.sum(axis = 0, skipna = True)
    b=result_pred_train.sum(axis = 0, skipna = True)


    dataframe=pd.DataFrame(dfs)
    dataset=dataframe.values

    train_size = int(len(dataset) * data_partition)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size], dataset[train_size:len(dataset)]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)

    sc_X = StandardScaler()
    sc_y = StandardScaler()
    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()


    trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
    testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))

    numpy.random.seed(1234)
    import tensorflow as tf
    tf.random.set_seed(1234)

    y1=pd.DataFrame(y1)
    y=pd.DataFrame(y)

    y_test= sc_y.inverse_transform (y1)
    y_train= sc_y.inverse_transform (y)


    a= pd.DataFrame(a)
    y_test= pd.DataFrame(y_test)



   #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,a)
    rmse= sqrt(mean_squared_error(y_test,a))
    mae=metrics.mean_absolute_error(y_test,a)

    return mape,rmse,mae


# In[32]:


def proposed_method(datass,look_back,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer):

    from PyEMD import CEEMDAN


    dfs=datass
    s = dfs.values

    emd = CEEMDAN(epsilon=0.05)
    emd.noise_seed(12345)

    IMFs = emd(s)


    full_imf=pd.DataFrame(IMFs)
    data_imf=full_imf.T



    pred_test=[]
    test_ori=[]
    pred_train=[]
    train_ori=[]


    n_imf=len(data_imf.columns)

    k=list(range(1,n_imf))
    m=[0]


    for i in m:

        datasetss2=pd.DataFrame(data_imf[i])
        datasets=datasetss2.values
        train_size = int(len(datasets) * data_partition)
        test_size = len(datasets) - train_size
        train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

        trainX, trainY = create_dataset(train, look_back)
        testX, testY = create_dataset(test, look_back)
        X_train=pd.DataFrame(trainX)
        Y_train=pd.DataFrame(trainY)
        X_test=pd.DataFrame(testX)
        Y_test=pd.DataFrame(testY)
        sc_X = StandardScaler()
        sc_y = StandardScaler()
        X= sc_X.fit_transform(X_train)
        y= sc_y.fit_transform(Y_train)
        X1= sc_X.fit_transform(X_test)
        y1= sc_y.fit_transform(Y_test)
        y=y.ravel()
        y1=y1.ravel()

        import numpy

        trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
        testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))

        numpy.random.seed(1234)
        import tensorflow as tf
        tf.random.set_seed(1234)


        from sklearn.ensemble import RandomForestRegressor


        grid = RandomForestRegressor(max_features=max_features)
        grid.fit(X,y)
        y_pred_train= grid.predict(X)
        y_pred_test= grid.predict(X1)

        y_pred_test=pd.DataFrame(y_pred_test)
        y_pred_train=pd.DataFrame(y_pred_train)

        y1=pd.DataFrame(y1)
        y=pd.DataFrame(y)

        y_test= sc_y.inverse_transform (y1)
        y_train= sc_y.inverse_transform (y)

        y_pred_test1= sc_y.inverse_transform (y_pred_test)
        y_pred_train1= sc_y.inverse_transform (y_pred_train)


        pred_test.append(y_pred_test1)
        test_ori.append(y_test)
        pred_train.append(y_pred_train1)
        train_ori.append(y_train)




    for i in k:

        datasetss2=pd.DataFrame(data_imf[i])
        datasets=datasetss2.values
        train_size = int(len(datasets) * data_partition)
        test_size = len(datasets) - train_size
        train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

        trainX, trainY = create_dataset(train, look_back)
        testX, testY = create_dataset(test, look_back)
        X_train=pd.DataFrame(trainX)
        Y_train=pd.DataFrame(trainY)
        X_test=pd.DataFrame(testX)
        Y_test=pd.DataFrame(testY)
        sc_X = StandardScaler()
        sc_y = StandardScaler()
        X= sc_X.fit_transform(X_train)
        y= sc_y.fit_transform(Y_train)
        X1= sc_X.fit_transform(X_test)
        y1= sc_y.fit_transform(Y_test)
        y=y.ravel()
        y1=y1.ravel()

        import numpy

        trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
        testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))

        numpy.random.seed(1234)
        import tensorflow as tf
        tf.random.set_seed(1234)


        from keras.models import Sequential
        from keras.layers.core import Dense, Dropout, Activation
        from keras.layers import LSTM

        neuron=neuron

        model = Sequential()
        model.add(LSTM(units = neuron,input_shape=(trainX.shape[1], trainX.shape[2])))
        model.add(Dense(1))
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(loss='mse',optimizer=optimizer)


        numpy.random.seed(1234)


     # Fitting the RNN to the Training set
        model.fit(trainX, y, epochs = epoch, batch_size = batch_size,verbose=0)


        # make predictions
        y_pred_train = model.predict(trainX)
        y_pred_test = model.predict(testX)

        # make predictions


        y_pred_test= numpy.array(y_pred_test).ravel()
        y_pred_test=pd.DataFrame(y_pred_test)
        y1=pd.DataFrame(y1)
        y=pd.DataFrame(y)
        y_pred_train= numpy.array(y_pred_train).ravel()
        y_pred_train=pd.DataFrame(y_pred_train)


        y_test= sc_y.inverse_transform (y1)
        y_train= sc_y.inverse_transform (y)

        y_pred_test1= sc_y.inverse_transform (y_pred_test)
        y_pred_train1= sc_y.inverse_transform (y_pred_train)


        pred_test.append(y_pred_test1)
        test_ori.append(y_test)
        pred_train.append(y_pred_train1)
        train_ori.append(y_train)

    result_pred_test= pd.DataFrame.from_records(pred_test)
    result_pred_train= pd.DataFrame.from_records(pred_train)


    a=result_pred_test.sum(axis = 0, skipna = True)
    b=result_pred_train.sum(axis = 0, skipna = True)


    dataframe=pd.DataFrame(dfs)
    dataset=dataframe.values

    train_size = int(len(dataset) * data_partition)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size], dataset[train_size:len(dataset)]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)

    sc_X = StandardScaler()
    sc_y = StandardScaler()
    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()


    trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
    testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))

    numpy.random.seed(1234)
    import tensorflow as tf
    tf.random.set_seed(1234)

    y1=pd.DataFrame(y1)
    y=pd.DataFrame(y)
    y_test= sc_y.inverse_transform (y1)
    y_train= sc_y.inverse_transform (y)


    a= pd.DataFrame(a)
    y_test= pd.DataFrame(y_test)


    import numpy as np


   #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,a)
    rmse= sqrt(mean_squared_error(y_test,a))
    mae=metrics.mean_absolute_error(y_test,a)

    return mape,rmse,mae,a,y_test


## Import parameter settings

## Run the experiments
### Run this following cell will train and test the proposed method and other benchmark methods on Primary Classroom Dataset

In [9]:
#Linear Regression

start_time = time.time()
lr_primclass=lr_model(datas_primclass,hours,data_partition)
lr_time_primclass=time.time() - start_time
print("--- %s seconds - Linear Regression- primclass ---" % (lr_time_primclass))

#Support Vector Regression
start_time = time.time()
svr_primclass=svr_model(datas_primclass,hours,data_partition)
svr_time_primclass=time.time() - start_time
print("--- %s seconds - Support Vector Regression- primclass ---" % (svr_time_primclass))


#ANN
start_time = time.time()
ann_primclass=ann_model(datas_primclass,hours,data_partition)
ann_time_primclass=time.time() - start_time
print("--- %s seconds - ANN- primclass ---" % (ann_time_primclass))

#random forest
start_time = time.time()
rf_primclass=rf_model(datas_primclass,hours,data_partition,max_features)
rf_time_primclass=time.time() - start_time
print("--- %s seconds - Random Forest- primclass ---" % (rf_time_primclass))

#LSTM
start_time = time.time()
lstm_primclass=lstm_model(datas_primclass,hours,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer)
lstm_time_primclass=time.time() - start_time
print("--- %s seconds - lstm- primclass ---" % (lstm_time_primclass))


#CEEMDAN RF
start_time = time.time()
ceemdan_rf_primclass=hybrid_ceemdan_rf(dfs_primclass,hours,data_partition,max_features)
ceemdan_rf_time_primclass=time.time() - start_time
print("--- %s seconds - ceemdan_rf- primclass ---" % (ceemdan_rf_time_primclass))

#CEEMDAN LSTM
start_time = time.time()
ceemdan_lstm_primclass=hybrid_ceemdan_lstm(dfs_primclass,hours,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer)
ceemdan_lstm_time_primclass=time.time() - start_time
print("--- %s seconds - ceemdan_lstm- primclass ---" % (ceemdan_lstm_time_primclass))


#proposed method
start_time = time.time()
proposed_method_primclass=proposed_method(dfs_primclass,hours,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer)
proposed_method_time_primclass=time.time() - start_time
print("--- %s seconds - proposed_method- primclass ---" % (proposed_method_time_primclass))



--- 0.23948955535888672 seconds - Linear Regression- primclass ---
--- 0.5788593292236328 seconds - Support Vector Regression- primclass ---
--- 4.8902857303619385 seconds - ANN- primclass ---
--- 3.5779764652252197 seconds - Random Forest- primclass ---
14/14 [==============================] - 0s 3ms/step
--- 11.122275590896606 seconds - lstm- primclass ---
--- 48.00813961029053 seconds - ceemdan_rf- primclass ---
14/14 [==============================] - 0s 2ms/step
--- 152.56399178504944 seconds - ceemdan_lstm- primclass ---
14/14 [==============================] - 0s 2ms/step
--- 148.50262808799744 seconds - proposed_method- primclass ---


## Summarize of experimental results with running time
### Run this following cell will summarize the result and generate output used in Section 4.4 (Table 3) for Primary Classroom dataset

In [10]:
running_time_primclass=pd.DataFrame([lr_time_primclass,svr_time_primclass,ann_time_primclass,
                                   rf_time_primclass,lstm_time_primclass,ceemdan_rf_time_primclass,
                                   ceemdan_lstm_time_primclass,proposed_method_time_primclass])
running_time_primclass=running_time_primclass.T
running_time_primclass.columns=['LR','SVR','ANN','RF','LSTM','CEEMDAN RF','CEEMDAN LSTM','Proposed Method']


proposed_method_primclass_df=proposed_method_primclass[0:3]
result_primclass=pd.DataFrame([lr_primclass,svr_primclass,ann_primclass,rf_primclass,lstm_primclass,ceemdan_rf_primclass,
                    ceemdan_lstm_primclass,proposed_method_primclass_df])
result_primclass=result_primclass.T
result_primclass.columns=['LR','SVR','ANN','RF','LSTM','CEEMDAN RF','CEEMDAN LSTM','Proposed Method']
primclass_summary=pd.concat([result_primclass,running_time_primclass],axis=0)

primclass_summary.set_axis(['MAPE(%)', 'RMSE','MAE','running time (s)'], axis='index')

primclass_summary.style.set_caption("primclass Results")

index = primclass_summary.index
index.name = "Primary Classroom results"
primclass_summary

,LR,SVR,ANN,RF,LSTM,CEEMDAN RF,CEEMDAN LSTM,Proposed Method
Primary Classroom results,,,,,,,,
0,15.508401,14.669799,14.463561,16.823501,16.300041,11.120485,8.320834,7.333805
1,1.040299,1.150338,0.902013,1.269686,0.952575,0.745849,0.518667,0.464208
2,0.650500,0.646708,0.577659,0.732776,0.620859,0.458639,0.334588,0.302695
0,0.239490,0.578859,4.890286,3.577976,11.122276,48.008140,152.563992,148.502628


In [ ]:
#export table to png
#dfi.export(primclass_summary,"primclass_summary_table.png")

## Calculate percentage improvement
### Run this following cell will calculate percentage improvement and generate output used in Section 4.4 (Table 4) for Primary Classroom dataset

In [11]:
pMAPE_LR_vs_Proposed_primclass=((lr_primclass[0]-proposed_method_primclass[0])/lr_primclass[0])*100
pRMSE_LR_vs_Proposed_primclass=((lr_primclass[1]-proposed_method_primclass[1])/lr_primclass[1])*100
pMAE_LR_vs_Proposed_primclass=((lr_primclass[2]-proposed_method_primclass[2])/lr_primclass[2])*100

pMAPE_SVR_vs_Proposed_primclass=((svr_primclass[0]-proposed_method_primclass[0])/svr_primclass[0])*100
pRMSE_SVR_vs_Proposed_primclass=((svr_primclass[1]-proposed_method_primclass[1])/svr_primclass[1])*100
pMAE_SVR_vs_Proposed_primclass=((svr_primclass[2]-proposed_method_primclass[2])/svr_primclass[2])*100

pMAPE_ANN_vs_Proposed_primclass=((ann_primclass[0]-proposed_method_primclass[0])/ann_primclass[0])*100
pRMSE_ANN_vs_Proposed_primclass=((ann_primclass[1]-proposed_method_primclass[1])/ann_primclass[1])*100
pMAE_ANN_vs_Proposed_primclass=((ann_primclass[2]-proposed_method_primclass[2])/ann_primclass[2])*100

pMAPE_RF_vs_Proposed_primclass=((rf_primclass[0]-proposed_method_primclass[0])/rf_primclass[0])*100
pRMSE_RF_vs_Proposed_primclass=((rf_primclass[1]-proposed_method_primclass[1])/rf_primclass[1])*100
pMAE_RF_vs_Proposed_primclass=((rf_primclass[2]-proposed_method_primclass[2])/rf_primclass[2])*100

pMAPE_LSTM_vs_Proposed_primclass=((lstm_primclass[0]-proposed_method_primclass[0])/lstm_primclass[0])*100
pRMSE_LSTM_vs_Proposed_primclass=((lstm_primclass[1]-proposed_method_primclass[1])/lstm_primclass[1])*100
pMAE_LSTM_vs_Proposed_primclass=((lstm_primclass[2]-proposed_method_primclass[2])/lstm_primclass[2])*100

pMAPE_ceemdan_rf_vs_Proposed_primclass=((ceemdan_rf_primclass[0]-proposed_method_primclass[0])/ceemdan_rf_primclass[0])*100
pRMSE_ceemdan_rf_vs_Proposed_primclass=((ceemdan_rf_primclass[1]-proposed_method_primclass[1])/ceemdan_rf_primclass[1])*100
pMAE_ceemdan_rf_vs_Proposed_primclass=((ceemdan_rf_primclass[2]-proposed_method_primclass[2])/ceemdan_rf_primclass[2])*100


pMAPE_ceemdan_lstm_vs_Proposed_primclass=((ceemdan_lstm_primclass[0]-proposed_method_primclass[0])/ceemdan_lstm_primclass[0])*100
pRMSE_ceemdan_lstm_vs_Proposed_primclass=((ceemdan_lstm_primclass[1]-proposed_method_primclass[1])/ceemdan_lstm_primclass[1])*100
pMAE_ceemdan_lstm_vs_Proposed_primclass=((ceemdan_lstm_primclass[2]-proposed_method_primclass[2])/ceemdan_lstm_primclass[2])*100


df_PI_primclass=[[pMAPE_LR_vs_Proposed_primclass,pMAPE_SVR_vs_Proposed_primclass,pMAPE_ANN_vs_Proposed_primclass,
                pMAPE_RF_vs_Proposed_primclass,pMAPE_LSTM_vs_Proposed_primclass,pMAPE_ceemdan_rf_vs_Proposed_primclass,
                pMAPE_ceemdan_lstm_vs_Proposed_primclass],
                [pRMSE_LR_vs_Proposed_primclass,pRMSE_SVR_vs_Proposed_primclass,pRMSE_ANN_vs_Proposed_primclass,
                pRMSE_RF_vs_Proposed_primclass,pRMSE_LSTM_vs_Proposed_primclass,pRMSE_ceemdan_rf_vs_Proposed_primclass,
                pRMSE_ceemdan_lstm_vs_Proposed_primclass],
                [pMAE_LR_vs_Proposed_primclass,pMAE_SVR_vs_Proposed_primclass,pMAE_ANN_vs_Proposed_primclass,
                pMAE_RF_vs_Proposed_primclass,pMAE_LSTM_vs_Proposed_primclass,pMAE_ceemdan_rf_vs_Proposed_primclass,
                pMAE_ceemdan_lstm_vs_Proposed_primclass]]

PI_primclass=pd.DataFrame(df_PI_primclass, columns=["Proposed Method vs.LR", "Proposed Method vs.SVR"," Proposed Method vs.ANN",
                                      "Proposed Method vs.RF","Proposed Method vs.LSTM","Proposed Method vs.CEEMDAN RF",
                                      "Proposed Method vs. CEEMDAN LSTM"])
PI_primclass= PI_primclass.round(decimals = 2)
PI_primclass.set_axis(['MAPE(%)', 'RMSE','MAE'], axis='index')
PI_primclass.style.set_caption("Percentage Improvement-primary classroom Building")

index = PI_primclass.index
index.name = "Percentage Improvement primclass"
PI_primclass

,Proposed Method vs.LR,Proposed Method vs.SVR,Proposed Method vs.ANN,Proposed Method vs.RF,Proposed Method vs.LSTM,Proposed Method vs.CEEMDAN RF,Proposed Method vs. CEEMDAN LSTM
Percentage Improvement primclass,,,,,,,
0,52.71,50.01,49.29,56.41,55.01,34.05,11.86
1,55.38,59.65,48.54,63.44,51.27,37.76,10.50
2,53.47,53.19,47.60,58.69,51.25,34.00,9.53


In [ ]:
#export table to png
#dfi.export(PI_primclass,"PI_primclass_table.png")